In [1]:
from langchain_community.llms import Ollama
from crewai import Agent, Task, Crew, Process

In [2]:
# Word list from https://github.com/jbowens/codenames/blob/master/assets/words.txt
# Rules from https://czechgames.com/files/rules/codenames-rules-en.pdf

In [3]:
rules_filename = "codenames-rules-en.txt"
with open(rules_filename, "r") as f:
    rules = f.read()

In [4]:
len(rules.split())  # Number of words - I don't want to overload the context window

3148

In [5]:
model = Ollama(model="llama3")

In [6]:
# TODO: Is there a better wau to pass the rules?
#       Maybe cut the rules down to the necessary parts only, or edit the rules to only what the LLM sees?

spymaster = Agent(
    role="hint proposer",
    goal="""
        Using the list of words, provide a one-word hint and a number indicating the number of words that the hint points to.
        You want to provide a hint to your teammates so that they can guess as many of your team's words as possible in a single 
        turn. The more words that you are able to hint at with a single clue, the better, but the other players should still be 
        able to reasonably guess the words you are hinting at from your clue.
    """,
    backstory=f"""
        Here are the rules for the game Codenames:
        {rules}
        You will be playing as the spymaster, but instead of seeing the board, you will be provided a list of words corresponding 
        to each team. Your job is to give a hint to your teammates so that they can guess as many of your team's words as possible
        in a single turn. Note that the hint must be a single word, and the number must generally be a non-negative integer, though 
        the number can also be 'inf' for infinity, as indicated in the rules for the game. You should avoid providing hints that 
        would lead the players on your team to pick the assassin, words belonging to the other team, or neutral words. You cannot 
        give a hint that is a word or word part already included in the word list.
    """,
    verbose=True,
    allow_delegation=False,
    llm=model
)

In [7]:
import csv

words_filename = "words.csv"
with open(words_filename, "r") as f:
    words = [row[0].lower() for row in csv.reader(f, delimiter='\n')]

In [8]:
import numpy as np

np.random.seed(916)
chosen_words = np.random.choice(words, 25, replace=False)
chosen_words

array(['toast', 'windmill', 'ovation', 'hang', 'stick', 'spare', 'lunch',
       'hedge', 'key', 'cruise', 'baby-sitter', 'doctor', 'niece', 'spy',
       'invitation', 'protestant', 'bat', 'lightsaber', 'ivy', 'mark',
       'truck', 'pirate', 'can', 'nest', 'puzzle'], dtype='<U13')

In [9]:
assert len(chosen_words) == 25

team1_words = "\n".join(chosen_words[:9])
team2_words = "\n".join(chosen_words[9:17])
neutral_words = "\n".join(chosen_words[17:24])
assassin_word = chosen_words[24]

game_words = f"""
Team 1 Words:
{team1_words}

Team 2 Words:
{team2_words}

Neutral Words:
{neutral_words}

Assassin Word:
{assassin_word}
"""

In [10]:
print(game_words)


Team 1 Words:
toast
windmill
ovation
hang
stick
spare
lunch
hedge
key

Team 2 Words:
cruise
baby-sitter
doctor
niece
spy
invitation
protestant
bat

Neutral Words:
lightsaber
ivy
mark
truck
pirate
can
nest

Assassin Word:
puzzle



In [14]:
# TODO: These tasks should be specific instances, aka they should include word lists, etc.

team_number = 1

give_hint = Task(
    description=f"{game_words}\nFrom the above word list, provide a one-word hint for Team {team_number} and a number indicating "
                "the number of words the hint corresponds to.",
    agent=spymaster,
    expected_output="One word and a number in the format 'hint number'. For example, 'fruit 3', 'color 0', or 'animal inf'.",
)

In [15]:
crew = Crew(
    agents = [spymaster],
    tasks = [give_hint],
    verbose = 2,
    process = Process.sequential
)

2024-04-25 18:12:03,369 - 139705733189632 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [16]:
output = crew.kickoff()

 [DEBUG]: == Working Agent: hint proposer
 [INFO]: == Starting Task: 
Team 1 Words:
toast
windmill
ovation
hang
stick
spare
lunch
hedge
key

Team 2 Words:
cruise
baby-sitter
doctor
niece
spy
invitation
protestant
bat

Neutral Words:
lightsaber
ivy
mark
truck
pirate
can
nest

Assassin Word:
puzzle

From the above word list, provide a one-word hint for Team 1 and a number indicating the number of words the hint corresponds to.


> Entering new CrewAgentExecutor chain...


ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f0f6b351720>: Failed to establish a new connection: [Errno 111] Connection refused'))